In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

DEBUG = False
EPOCHS = 10


from baseline_curricula import upload, split_into_epochs, repeat_for_ten_epochs

In [ ]:
import config

In [ ]:
from baseline_curricula import validate_training_duration_limitation
from itertools import product
import config 

[validate_training_duration_limitation(dataset_name, "llama"+curriculum_name) for dataset_name, curriculum_name in product(config.datasets, config.influence_curricula)]



In [ ]:
from baseline_curricula import get_perplexity_ngram

for dataset_name in config.datasets:
    perplexity_df = get_perplexity_ngram(dataset_name)
    upload("perplexity_increasing.pt", repeat_for_ten_epochs(perplexity_df["perplexity"].sort_values(ascending=True).index.to_list()), dataset_name)


In [ ]:
from baseline_curricula import get_mattr


for dataset_name in config.datasets:
    mattr_df = get_mattr(dataset_name, n_proc=128)
 #   upload("perplexity_decreasing.pt", split_into_epochs(perplexity_df["perplexity"].sort_values(ascending=False).index.to_list()), dataset)
    upload("mattr_increasing.pt", repeat_for_ten_epochs(mattr_df["mattr"].sort_values(ascending=True).index.to_list()), dataset_name)


In [ ]:
from baseline_curricula import get_mattr
import torch


for dataset_name in config.datasets:
    
    torch.manual_seed(42)
    dataset = load_dataset(dataset_name)
    random_order = [torch.randperm(len(dataset["train"])) for _ in range(0,EPOCHS)]
    
    upload("random.pt", random_order, dataset_name)


In [ ]:
from baseline_curricula import validate_training_duration_limitation

In [ ]:


word_count = lambda d: {"words" : len(d["text"].split())}

WORDS_PER_STAGE = 2*10000000 # there are 5 stages: 2*10M words = 100M total seen during training
for dataset_name in config.datasets:
    
    
    df = load_dataset(dataset_name)["train"].map(word_count, num_proc=100).to_pandas()
    curriculum = []
    for stage_name, stage_df in df.groupby("stage"):
        print(stage_name)
        tokens_in_stage = 0
        curriculum_stage = []
        
        i = 0
        while tokens_in_stage < WORDS_PER_STAGE: # the dataset assumes each stage is repeated for multiple epochs. we will therefore run out of documents before reaching the alloted number of tokens. just re-shuffle and continue
                                                 #    we double-check the total number of tokens seen trough the curriculum below
            stage_df_shuffled = stage_df.sample(frac=1, random_state=42+i)
            for idx, row in stage_df_shuffled.iterrows():            
                if (tokens_in_stage + row["words"]) <= WORDS_PER_STAGE:
                    curriculum_stage.append(idx)
                    tokens_in_stage += row["words"]
                else:
                    tokens_in_stage += row["words"] # so that outer loop breaks
                    break
                
            print(i,tokens_in_stage)
            i+=1
        curriculum.extend(torch.chunk(torch.tensor(curriculum_stage), 2)) # so that there are 2 epochs per stage
    validate_training_duration_limitation(dataset_name, curriculum)

    upload("source_difficulty.pt", curriculum, dataset_name)


In [ ]:
from baseline_curricula import validate_training_duration_limitation
from itertools import product
import config 

[validate_training_duration_limitation(dataset_name, curriculum_name) for dataset_name, curriculum_name in [product(config.datasets, config.baseline_curricula)]]

